In [1]:
from sklearn.impute import SimpleImputer
from sklearn import linear_model
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("2008_limpia.csv")
df=data.sample(frac=1).head(10000)
X=df[['AirTime', "Distance", "TaxiIn", "TaxiOut", "DepDelay", "ArrDelay", "CarrierDelay"]]
X

,AirTime,Distance,TaxiIn,TaxiOut,DepDelay,ArrDelay,CarrierDelay
7496,112.0,719,13.0,19.0,-2.0,-4.0,NaN
7186,134.0,1236,4.0,13.0,25.0,1.0,NaN
4005,73.0,508,6.0,8.0,-14.0,-32.0,NaN
3936,296.0,2602,5.0,15.0,-6.0,-14.0,NaN
1751,41.0,213,8.0,87.0,2.0,62.0,0.0
...,...,...,...,...,...,...,...
7805,53.0,325,8.0,14.0,-1.0,-3.0,NaN
1666,246.0,1747,8.0,21.0,-3.0,-3.0,NaN
9653,85.0,566,4.0,28.0,-4.0,1.0,NaN
4829,144.0,1034,10.0,15.0,-7.0,-21.0,NaN


In [4]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(X)
X_imp = imp.transform(X)
X_imp

array([[ 1.12000000e+02,  7.19000000e+02,  1.30000000e+01, ...,
        -2.00000000e+00, -4.00000000e+00,  1.56550957e+01],
       [ 1.34000000e+02,  1.23600000e+03,  4.00000000e+00, ...,
         2.50000000e+01,  1.00000000e+00,  1.56550957e+01],
       [ 7.30000000e+01,  5.08000000e+02,  6.00000000e+00, ...,
        -1.40000000e+01, -3.20000000e+01,  1.56550957e+01],
       ...,
       [ 8.50000000e+01,  5.66000000e+02,  4.00000000e+00, ...,
        -4.00000000e+00,  1.00000000e+00,  1.56550957e+01],
       [ 1.44000000e+02,  1.03400000e+03,  1.00000000e+01, ...,
        -7.00000000e+00, -2.10000000e+01,  1.56550957e+01],
       [ 2.90000000e+01,  1.63000000e+02,  5.00000000e+00, ...,
        -2.00000000e+00, -1.00000000e+00,  1.56550957e+01]])

In [5]:
pd.DataFrame(X_imp, columns=X.columns)

,AirTime,Distance,TaxiIn,TaxiOut,DepDelay,ArrDelay,CarrierDelay
0,112.0,719.0,13.0,19.0,-2.0,-4.0,15.655096
1,134.0,1236.0,4.0,13.0,25.0,1.0,15.655096
2,73.0,508.0,6.0,8.0,-14.0,-32.0,15.655096
3,296.0,2602.0,5.0,15.0,-6.0,-14.0,15.655096
4,41.0,213.0,8.0,87.0,2.0,62.0,0.000000
...,...,...,...,...,...,...,...
9995,53.0,325.0,8.0,14.0,-1.0,-3.0,15.655096
9996,246.0,1747.0,8.0,21.0,-3.0,-3.0,15.655096
9997,85.0,566.0,4.0,28.0,-4.0,1.0,15.655096
9998,144.0,1034.0,10.0,15.0,-7.0,-21.0,15.655096


Esto no se debe de hacer mucho pues perdemos varianza

In [7]:
X.corr()

,AirTime,Distance,TaxiIn,TaxiOut,DepDelay,ArrDelay,CarrierDelay
AirTime,1.000000,0.982912,0.092871,0.127214,0.036886,0.054973,0.004257
Distance,0.982912,1.000000,0.083129,0.109746,0.037176,0.027182,0.033128
TaxiIn,0.092871,0.083129,1.000000,0.050486,0.045692,0.148195,-0.068793
TaxiOut,0.127214,0.109746,0.050486,1.000000,0.108320,0.355661,-0.087627
DepDelay,0.036886,0.037176,0.045692,0.108320,1.000000,0.922047,0.463492
ArrDelay,0.054973,0.027182,0.148195,0.355661,0.922047,1.000000,0.419241
CarrierDelay,0.004257,0.033128,-0.068793,-0.087627,0.463492,0.419241,1.000000


Los que influyen más en la CarrierDelay son la Dep y la Arr, así que hacemos una regresión con estas para llenas los espacios faltantes de la carrier

In [11]:
X = X.dropna(subset=["DepDelay", "ArrDelay"])
newX= X[["DepDelay", "ArrDelay", "CarrierDelay"]]
newX_train = newX[np.isnan(X.CarrierDelay)==False]

regr = linear_model.LinearRegression()
regr.fit(newX_train[["DepDelay", "ArrDelay"]], newX_train.CarrierDelay)

LinearRegression()

In [12]:
regr.coef_

array([ 0.31132127, -0.01715413])

In [13]:
#Aquí lo que hacemos es tomar los valores de los datos que no tengan valor para
# predecir el valor de esos datos.
valores_imputacion = newX[np.isnan(X.CarrierDelay)]
valores_imputacion = valores_imputacion[["DepDelay", "ArrDelay"]]

In [14]:
nuevos_valores = regr.predict(valores_imputacion)

In [15]:
X.loc[np.isnan(X.CarrierDelay), "CarrierDelay"] = nuevos_valores

In [18]:
X

,AirTime,Distance,TaxiIn,TaxiOut,DepDelay,ArrDelay,CarrierDelay
7496,112.0,719,13.0,19.0,-2.0,-4.0,0.949473
7186,134.0,1236,4.0,13.0,25.0,1.0,9.269377
4005,73.0,508,6.0,8.0,-14.0,-32.0,-2.306066
3936,296.0,2602,5.0,15.0,-6.0,-14.0,-0.124270
1751,41.0,213,8.0,87.0,2.0,62.0,0.000000
...,...,...,...,...,...,...,...
7805,53.0,325,8.0,14.0,-1.0,-3.0,1.243641
1666,246.0,1747,8.0,21.0,-3.0,-3.0,0.620998
9653,85.0,566,4.0,28.0,-4.0,1.0,0.241060
4829,144.0,1034,10.0,15.0,-7.0,-21.0,-0.315513
